In [1]:
# installing boto 3
! pip install boto3

   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   ------ --------------------------------- 2.1/13.2 MB 14.7 MB/s eta 0:00:01
   ---------------- ----------------------- 5.5/13.2 MB 15.2 MB/s eta 0:00:01
   -------------------------- ------------- 8.7/13.2 MB 15.3 MB/s eta 0:00:01
   ------------------------------------ --- 12.1/13.2 MB 15.7 MB/s eta 0:00:01
   ---------------------------------------- 13.2/13.2 MB 14.8 MB/s eta 0:00:00


# Setup Connection Information

In [3]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import boto3
import shutil
import glob
import datetime as dt
from datetime import date, datetime, timedelta


bucket_name = 'Your Teams Bucket Name'

enter_aws_access_key_id ='Access Key' 

enter_aws_secret_access_key ='Secret Access Key'

#NO SPACE between group & number
group_number = 'group13' 

# path to the folder to save NO SAPCE in folder name
save_folder_path = os.getcwd()

# the folder names in the data folder your group members names as apear on AWS S3
#for example
participant_list = ['ParticipantId-1']

# Downloading the data

In [ ]:
s3 = boto3.resource('s3',
                 aws_access_key_id= enter_aws_access_key_id,
                 aws_secret_access_key= enter_aws_secret_access_key)


#     ## Bucket to use
bucket = s3.Bucket(bucket_name)


#get participant list
def get_participant_list(bucket_input):
    client = boto3.client('s3',
                     aws_access_key_id= enter_aws_access_key_id,
                     aws_secret_access_key= enter_aws_secret_access_key)
    result = client.list_objects(Bucket=bucket_input, Prefix='data/', Delimiter='/')
    part_list = []
    for o in result.get('CommonPrefixes'):
        part_list.append(o.get('Prefix').split('/')[1])
    return part_list

#update this to the correct format


#path of the files
file_list=[]


def get_objects(part_name):
     for obj in bucket.objects.filter(Delimiter='/', Prefix=f'data/{part_name}/'):
         file_list.append(obj.key)

for i in range(len(participant_list)):
     get_objects(participant_list[i])

def download_dir(prefix, local, bucket):
    client = boto3.client('s3',
                 aws_access_key_id= enter_aws_access_key_id,
                 aws_secret_access_key= enter_aws_secret_access_key)
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)


for i in range(len(participent_list)):
    download_dir(('data/'+participent_list[i]),save_folder_path,bucket_name)


# Creating the data set and output CSV

In [10]:
#Creating the dataset function
def create_combined_dataset(path_to_raw_data_folder):
    data = [pd.read_json(f'{path_to_raw_data_folder}/{i}') for i in os.listdir(path_to_raw_data_folder) if i.endswith('.gz')]
    return pd.concat(data)

# Function to fix timestamp from UTC to EST
def fix_time(a):
    b = pd.Timestamp(a)
    c = b.tz_convert("America/New_York")
    d = pd.to_datetime(c)
    e = d.to_pydatetime().strftime('%Y-%m-%d %H:%M:%S')
    return(e)


try:
    os.mkdir(save_folder_path+'/data/allfiles')
except:
    pass
#Move all the results into a single folder
for j in range(len(participent_list)):
    source_dir = f'{save_folder_path}/data/{participent_list[j]}'

    target_dir = save_folder_path +'/data/allfiles'

    file_names = os.listdir(source_dir)

    for file_name in file_names:
        shutil.copy(os.path.join(source_dir, file_name), target_dir)

# find corrupt files, move them to a corrupt folder
zip_file_names = list(glob.glob(target_dir+"/*.gz"))
corrupt_files = []
ok_files_count=0
corrupt_file_count = 0
for i in range(len(zip_file_names)):
    try:
        pd.read_json(zip_file_names[i])
        ok_files_count += 1  
        #print(zip_file_names[i] +' ---> FILE OK')
    except:
        corrupt_files.append(zip_file_names[i])
        #print(zip_file_names[i]+' ---> CORRUPTED')
        corrupt_file_count += 1

################################ folders for files 
try:
    os.mkdir(save_folder_path+'/data/corrupt_file_folder')
except:
    pass
try:
    for i in range(len(corrupt_files)):
        shutil.move(f"{corrupt_files[i]}", f"{save_folder_path}/data/corrupt_file_folder")        
except:
    pass

df = create_combined_dataset(target_dir)        

df.rename(columns={'$type':'Type'},inplace=True) #fixing the name

df.rename(columns={'Level':'BatteryLevel'},inplace=True) #fixing the level name

df.rename(columns={'Accuracy':'LocationAccuracy'},inplace=True) #fixing the accuracy name

# this is a metadata column that can remove
try:
    df.drop('ProbeParticipation', axis=1, inplace=True)
except:
    pass


# If the column protocol name has the protocol name we remove it since every
# participent is in the same study
try:
    df= df[df['ProtocolName'].isnull()] #removing info row
except:
    pass

#
df.index = pd.RangeIndex(len(df.index)) #fixing the index
df.index.name = 'Row_id' #adding a new id row
df['Formatted_time'] = df['Timestamp'].apply(lambda x: fix_time(x)) # adding a new column changing UTC to EST
df['Formatted_time'] = pd.to_datetime(df['Formatted_time'] ) # fix data type


# remove extra
df['Type'] = df['Type'].replace(regex = {'Sensus.Participation':''})
df['Type'] = df['Type'].replace(regex = {' SensusiOS':''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Device.': ''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Location.': ''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Movement.':''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Network.': ''})
df['Type'] = df['Type'].replace(regex = {'Sensus.Probes.Context.': ''})
df['Type'] = df['Type'].replace(regex = {'Datum,':''})

# removing metadata rows
df = df[df['Type'] != 'Sensus.Heartbeat']
df = df[df['Type'] != 'Report']

### fixing the values for mysql
df['Timestamp'] = pd.to_datetime(df['Timestamp'],utc=True)
df['Formatted_time'] = pd.to_datetime(df['Formatted_time'] )

# fixing the order
try:
    df = df[['Type','Id','DeviceId','Timestamp','ProtocolId','BuildId','ParticipantId','DeviceManufacturer',
         'DeviceModel','OperatingSystem','TaggedEventId','TaggedEventTags','SensingAgentStateDescription',
         'LocalOffsetFromUTC','Decibels','BatteryLevel','AccessPointBSSID','Activity','Phase','State',
         'Confidence','Latitude','Longitude','LocationAccuracy','ProtocolName','Formatted_time']]
except:
    df = df[['Type','Id','DeviceId','Timestamp','ProtocolId','BuildId','ParticipantId','DeviceManufacturer',
         'DeviceModel','OperatingSystem','TaggedEventId','TaggedEventTags','SensingAgentStateDescription',
         'LocalOffsetFromUTC','Decibels','BatteryLevel','AccessPointBSSID','Activity','Phase','State',
         'Confidence','ProtocolName','Formatted_time']]

df.to_csv(f'{save_folder_path}/{group_number}_output.csv')

shutil.rmtree(f'{save_folder_path}/data/allfiles')
